<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Class_06_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

#### **Change your Runtime Now!**

For this lesson you must have a good **GPU** hardware accelerator like the `A-100`.

##### **Module 6: Advanced Topics**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)

### Module 6 Material

* Part 6.1: Reinforcement Learning
* Part 6.2: ONNX Runtime Environment
* **Part 6.3: Analysis of DICOM images with Pytorch**
* Part 6.4: Agentic AI (Revisited)

## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to /content/drive and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [ ]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    COLAB = False

You should see the following output except your GMAIL address should appear on the last line.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image01B.png)

If your GMAIL address does not appear your lesson will **not** be graded.

## Accelerated Run-time Check

You MUST run the following code cell to get credit for this class lesson. The code in this cell checks what hardware acceleration you are using. To run this lesson, you must be running a Graphics Processing Unit (GPU) such as the `A100`.

In [ ]:
# You must run this cell second

import torch

# Check for GPU
def check_colab_gpu():
    print("=== Colab GPU Check ===")

    # Check PyTorch
    pt_gpu = torch.cuda.is_available()
    print(f"PyTorch GPU available: {pt_gpu}")

    if pt_gpu:
        print(f"PyTorch device count: {torch.cuda.device_count()}")
        print(f"PyTorch current device: {torch.cuda.current_device()}")
        print(f"PyTorch device name: {torch.cuda.get_device_name()}")
        print("You are good to go!")

    else:
        print("No compatible device found")
        print("WARNING: You must run this assigment using either a GPU to earn credit")
        print("Change your RUNTIME now and start over!")

check_colab_gpu()


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image01E.png)

### Create Custom Function

Run the code in the next cell to create the custom timing function.

In [ ]:
# Simple function to print out elapsed time
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

### Install Packages

Run the cell below to install the various packages needed for this lesson.

In [ ]:
!pip install -q pydicom
!pip install -q dropblock

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image02E.png)

# **Pneumonia Detection from DICOM Image Analysis**

In this lesson we will introduce the following advanced techniques that have not been covered so far in this course:

* **1. DICOM Image analysis**

* **2. _Lazy_ Loading of image data**

* **3. Training Deep Neural Networks using `PyTorch`**

The images to be analyzed in this lesson are a subset of the NIH Chest‑X‑Ray‑8 (CXR‑8) dataset (https://data.nih.gov). This subset consists of `9937` images in **DICOM format** that are frontal-view chest-X-rays. Each image is a single‑channel (grayscale) image of roughly 1024 × 1024 pixels.

![__](https://biologicslab.co/BIO1173/images/class_06/DICOM_Images.png)
**Figure:** The 3 different image classes in the DICOM dataset: **A** `Normal`, **B** `No Opacity/Not Normal` and **C** `Opacity`. Image classes **A** and **B** were assigned the `Target` value `0`(no pneumonia) while class **C** was assigned the `Target` value `1` (presence of pneumonia).

The early detection and treatment of pneumonia is an important medical challenge. According to UNICEF, pneumonia and diarrhoea **kill 1.4 million children each year**, more than all other childhood illnesses combined.





## **DICOM Image Analysis**

**DICOM (Digital Imaging and Communications in Medicine)** is a standard for handling, storing, printing, and transmitting medical images and related information. It was developed by the National Electrical Manufacturers Association (NEMA) as an international standard for healthcare informatics.

#### **Key Features of DICOM**
- **Standardized Format**: Ensures that medical images can be exchanged between different systems without loss of image quality or clinical relevant information.
- **Metadata Storage**: Includes patient demographics, study details, imaging parameters, and other critical data in a structured format.
- **Interoperability**: Facilitates communication between various medical devices, software applications, and healthcare providers.

#### **Use Cases**
1. **Image Transfer**: DICOM is used to transfer images from one modality (e.g., MRI, CT) to another or between different hospitals.
2. **Storage and Retrieval**: For long-term storage of medical images in hospital networks, ensuring that images can be accessed when needed for review or consultation.
3. **Quality Control**: In diagnostic imaging departments, DICOM allows for the verification and monitoring of image quality before distribution to clinicians.
4. **Research and Education**: Used extensively in medical research and educational settings to analyze patient data across different studies and institutions.
5. **Clinical Diagnostics**: Critical for clinical diagnosis as it provides detailed information about the imaging procedure, which is crucial for treatment planning and follow-up.

#### **Importance for Students Learning Computational Biology**
1. **Understanding Clinical Context**: Learning DICOM helps students to understand the context of medical images and their relevance in patient care.
2. **Advanced Analysis**: Proficiency in handling DICOM images is essential for advanced analysis, such as image segmentation and machine learning applications in clinical diagnostics.
3. **Research Skills**: In biocomputational fields, working with large datasets including DICOM images is common. Understanding how to manage these data sets enhances research capabilities.
4. **Future Professional Development**: Knowledge of DICOM is becoming a standard requirement for many healthcare IT and bioinformatics positions, ensuring that students can meet industry standards post-graduation.

#### **Why Learning DICOM is Important for Clinical Diagnostic Analysis**
- **Accurate Diagnosis**: Understanding the metadata in DICOM images helps clinicians to make accurate diagnoses by reviewing imaging data in context with patient information.
- **Treatment Planning**: Accurate image interpretation allows physicians to plan effective treatments, which can significantly impact patient outcomes.
- **Interoperability and Integration**: In a healthcare environment where multiple systems are integrated, knowledge of DICOM ensures seamless communication between different medical devices and software applications.
- **Legal and Ethical Considerations**: Proper handling and storage of DICOM images comply with legal and ethical standards in healthcare data management.

### **Conclusion**
**DICOM images** play a pivotal role in modern healthcare, especially in clinical diagnostic analysis. For biocomputational college students, mastering the use of DICOM is not only beneficial for academic learning but also crucial for future professional success in the field of healthcare informatics and bioinformatics.


### Example 1: Download DICOM Image Data

This Python code in Example 1 downloads a zip file from a specified URL and save it to a local destination. The efficiency of this code comes from several aspects, primarily due to its design choices which optimize the process for both speed and resource usage:

1. **Streamed Download:** By using requests.get(url, stream=True), the data is streamed directly from the server without downloading the entire file into memory at once. This is particularly useful when dealing with large files where loading the whole file could be impractical or consume excessive memory.

2. **Chunk Size Optimization:** The parameter chunk_size (set to 8192 bytes by default) allows the code to write data in manageable chunks rather than all at once, which can improve performance on systems with limited memory or slow I/O operations.

3. **Progressive Download Notification:** The print statement "Downloading ... done" provides feedback indicating that the download process has started and completed successfully, giving users a clear indication of progress without additional user interaction.

4. **Error Handling:** r.raise_for_status() ensures that an HTTPError is raised if there was an issue with the request (e.g., 404 Not Found or 500 Server Error). This helps in debugging and ensuring that the script stops running when it encounters a problem, preventing potential corruption of partially downloaded files.

5. **File Writing:** The file is opened in binary mode ("wb") to ensure compatibility with various types of data (including images, binaries, etc.) and to properly handle all bytes being transferred without alteration.

Overall, this script is efficient because it handles large files efficiently by not loading them entirely into memory, allows for customization via parameters like chunk size, provides clear user feedback, includes error handling, and ensures that the file transfer respects data integrity through binary mode writing.

**TIME WARNING:** This is a large datafile so it will take about 2-3 min to download to your Colab notebook.

In [ ]:
# Example 1: Download DICOM Image Data

import requests
from pathlib import Path

# Configuration
URL = "https://biologicslab.co/BIO1173/data/"
ZIP_FILENAME = "pna_data.zip"

# Download the zip file (streamed, so it works with large files)
cwd          = Path.cwd()            # current working directory
zip_path     = cwd / ZIP_FILENAME
extract_dir  = cwd / zip_path.stem   # e.g., /pna_data

# Ensure the extraction directory exists
extract_dir.mkdir(parents=True, exist_ok=True)

print(f"Downloading {ZIP_FILENAME} to {zip_path}...", end='')
with requests.get(URL + ZIP_FILENAME, stream=True, timeout=30) as r:
    r.raise_for_status()           # will raise for 4xx/5xx
    with zip_path.open("wb") as f_out:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:               # filter out keep-alive new chunks
                f_out.write(chunk)
print("done")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image04E.png)

### Example 2: Extract DICOM Image Data

The code in the cell below unzips the DICOM image data and stores it in a new folder with the same name as the zip file.

In [ ]:
# Example 2: Extract DICOM Image Data

import zipfile
from pathlib import Path

# Set Clean Up Flag
clean_up = True  # Use lowercase for variable names

# Configuration
zip_filename = "pna_data.zip"

# Unzip the downloaded archive into a named directory
cwd = Path.cwd()  # current working directory
extract_dir = cwd / zip_filename.replace('.zip', '')  # e.g., /pna_data

# Ensure the extraction directory exists
extract_dir.mkdir(parents=True, exist_ok=True)

print(f"Unzipping {zip_filename} to {extract_dir}...", end='')
with zipfile.ZipFile(cwd / zip_filename, "r") as zf:
    zf.extractall(path=extract_dir)
print("done")

# Optional – delete the zip after extraction
def clean_up_zip(zip_path: Path) -> None:
    """Delete the zip file – only if you no longer need it."""
    zip_path.unlink()
    print(f"Removed temporary archive: {zip_path}... done")

# Optionally call the clean-up function
if clean_up:
    clean_up_zip(Path(cwd / zip_filename))  # Ensure path matches file name case
    print(f"Files have been successfully extracted to {extract_dir} and the Zip file deleted")
else:
    print(f"Files have been successfully extracted to {extract_dir}")


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image03E.png)

### Create Functions to Process DICOM Files

The code in the cell below uses a special package called `pydicom`. **Pydicom** is a Python package for specifically handling DICOM (Digital Imaging and Communications in Medicine) files. It allows you to read, write, modify, and create DICOM files using Python code. DICOM is a standard used in healthcare imaging that defines file formats and communications protocols for medical imaging devices and information systems.

Pydicom provides classes and functions to work with the data elements defined in DICOM standards, such as Patient Name, Study Date, Modality, and many others. It supports reading from and writing to DICOM files, handling pixel data (using NumPy arrays for efficient processing), and accessing metadata embedded within DICOM files.

The code is divided into a series of functions (e.g. `read_dicom_file(file_path:str`) that will be called later.

**Define the Path to the Data Root:**
```python
  # Path to the data root
  data_root = os.path.join('.', 'pna_data')
  print(f"data_root = {data_root}")
```
This line constructs a file path pointing to a directory named 'pna_data' located in the current working directory (.). The os.path.join function is used to join the current directory with the folder name, forming an absolute path like '/Users/username/project/./pna_data' on Unix-based systems or 'C:\\Users\\username\\project\\./pna_data' on Windows systems (depending on where your project and data are located).


In [ ]:
# Create Functions to Process DICOM Files

import numpy as np
import pandas as pd
import pydicom
import warnings
from matplotlib import pyplot as plt

# Global settings
warnings.filterwarnings('ignore')


#  Helper: read a single DICOM file
def read_dicom_file(file_path: str):
    """Read a DICOM file and extract image data + basic metadata."""
    ds = pydicom.dcmread(file_path)

    # Basic metadata
    metadata = {
        'filename': os.path.basename(file_path),
        'patient_name': getattr(ds, 'PatientName', 'Unknown'),
        'patient_id': getattr(ds, 'PatientID', 'Unknown'),
        'study_date': getattr(ds, 'StudyDate', 'Unknown'),
        'study_time': getattr(ds, 'StudyTime', 'Unknown'),
        'modality': getattr(ds, 'Modality', 'Unknown'),
        'manufacturer': getattr(ds, 'Manufacturer', 'Unknown'),
        'institution_name': getattr(ds, 'InstitutionName', 'Unknown'),
        'series_description': getattr(ds, 'SeriesDescription', 'Unknown'),
        'bits_allocated': getattr(ds, 'BitsAllocated', 'Unknown'),
        'rows': getattr(ds, 'Rows', 'Unknown'),
        'columns': getattr(ds, 'Columns', 'Unknown'),
        'pixel_spacing': getattr(ds, 'PixelSpacing', 'Unknown')
    }

    # Image data
    if hasattr(ds, 'pixel_array'):
        image_array = ds.pixel_array

        # Normalise to 0‑255 if needed
        if image_array.dtype != np.uint8:
            image_array = ((image_array - image_array.min()) /
                           (image_array.max() - image_array.min()) * 255).astype(np.uint8)

        metadata['image_available'] = True
        metadata['image_shape'] = image_array.shape
    else:
        metadata['image_available'] = False
        metadata['image_shape'] = 'No image data'

    return ds, metadata

#  Helper: fast drop‑check
def is_file_dropped(file_path: str) -> bool:
    """
    Quick guard that tells us whether a DICOM file is already
    missing / unreadable.
    """
    if not os.path.isfile(file_path):
        return True

    if os.path.getsize(file_path) == 0:
        return True

    try:
        pydicom.dcmread(file_path, stop_before_pixels=True)
    except Exception:
        return True

    return False

# ---- Load & preprocess – merge CSVs, keep only valid DICOM rows
import os
import pandas as pd
import pathlib

def load_and_preprocess_data(
    data_dir: str = '.',
    log_dropped: bool = True,
    max_files: int | None = None,
    seed: int = 42
):
    """
    Load the two CSVs, merge on patient ID, filter out rows that don't have a
    valid DICOM file and optionally subsample the result.

    Returns:
        image_df (pd.DataFrame):  DataFrame that contains a new column
                                 `file_path` pointing to the DICOM file
                                 and the original merged columns.
        dropped_ids (list[str]):  Patient IDs that were dropped.
    """
    # --- 1. Resolve the path & sanity‑check the CSVs
    data_path   = pathlib.Path(data_dir).expanduser().resolve()
    info_csv    = data_path / 'pna_detailed_class_info.csv'
    labels_csv  = data_path / 'pna_train_labels.csv'

    for fp in (info_csv, labels_csv):
        if not fp.is_file():
            raise FileNotFoundError(
                f"Required file not found: {fp}\n"
                f"Ensure `data_dir` points to the folder containing both CSVs."
            )

    # --- 2. Load the CSVs
    info_df   = pd.read_csv(info_csv)
    labels_df = pd.read_csv(labels_csv)

    # --- 3. Merge on patient ID
    info_id_col   = 'patientId'
    labels_id_col = 'patientId'

    merged_df = pd.merge(
        info_df,
        labels_df,
        left_on=info_id_col,
        right_on=labels_id_col,
        how='inner'
    )

    # --- 4. Keep only rows that actually have a readable DICOM
    dicom_dir   = data_path / 'pna_train_images'
    valid_rows  = []
    dropped_ids = []

    for idx, row in merged_df.iterrows():
        patient_id = row[info_id_col]
        dicom_file = dicom_dir / f"{patient_id}.dcm"

        if is_file_dropped(dicom_file):
            dropped_ids.append(patient_id)
        else:
            valid_rows.append(idx)

    image_df = merged_df.loc[valid_rows].copy()
    if log_dropped:
        print(f"Dropped {len(dropped_ids)} rows (no valid DICOM).")

    # --- 5. (Optional) Random subsample
    if max_files is not None:
        if max_files == -1:
            # Do nothing; use the full dataset
            pass
        elif len(image_df) > max_files:
            image_df = image_df.sample(n=max_files, random_state=seed).reset_index(drop=True)
            if log_dropped:
                  print(f"Randomly subsampled to {max_files} rows (seed={seed}).")

    # --- 6. Add `file_path` column
    #   This mirrors the original behaviour – the rest of the code can still
    #   reference `image_df['file_path']` exactly as before.
    image_df['file_path'] = image_df['patientId'].apply(
        lambda pid: str(dicom_dir / f"{pid}.dcm")
    )

    # print(f"Filtered DataFrame shape (with valid DICOM files): {image_df.shape}")
    return image_df, dropped_ids

#  Update load_and_preprocess_data to return only file‑paths + labels
def load_file_paths_and_labels(image_df, max_samples=None):
    """
    Return two lists: `file_paths` and `labels`.  The function keeps the
    *original* logic that filtered out NaNs / duplicated rows – we simply
    strip it down to the very few objects that the new lazy loader needs.
    """
    file_paths = image_df["file_path"].tolist()
    labels = image_df["label"].tolist()

    if max_samples is not None:
        file_paths = file_paths[:max_samples]
        labels = labels[:max_samples]

    return file_paths, labels


#  Show a single DICOM image (for sanity checks)
def display_dicom_image(file_path: str, figsize: tuple = (5, 5)):
    """Show a single DICOM image with proper orientation."""
    ds = pydicom.dcmread(file_path)
    if hasattr(ds, 'pixel_array'):
        img = ds.pixel_array
        if getattr(ds, 'PhotometricInterpretation', None) == 'MONOCHROME1':
            img = np.max(img) - img

        plt.figure(figsize=figsize)
        plt.imshow(img, cmap='gray')
        plt.title(ds.SOPClassUID)
        plt.axis('off')
        plt.show()
    else:
        print("This DICOM has no pixel data.")

# Path to the data root
data_root = os.path.join('.', 'pna_data')
print(f"data_root = {data_root}")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image05E.png)

### Example 3: Display a DICOM Image

The code in the cell below uses the function `display_dicom_image()` to display one of three image choices based on the `user input`. The code catches errors and continues to prompt the user for an image choice until a valid name is entered. Your input choices are "Normal", "No Lung Opacity/Not Normal" and "Lung Opacity". Only the lung images that are labled as "Lung Opacity" exhibit signs of pneumonia.  

In [ ]:
# Example 3: Display DICOM image

# Extract path names
zip_filename=ZIP_FILENAME
folder_root =  zip_filename.replace('.zip', '')
data_root = zip_filename.replace('_data.zip', '')
image_folder = data_root+"_train_images/"

# Generate path to image folder
path ="./"+folder_root+"/"+image_folder

print(path)

image_paths = {
    "Normal": path+"c1edf42b-5958-47ff-a1e7-4f23d99583ba.dcm",
    "No Lung Opacity/Not Normal": path+"00322d4d-1c29-4943-afc9-b6754be640eb.dcm",
    "Lung Opacity": path+"c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8.dcm"
}

# Set display size
figsize = (6, 6)

# Ask the user which class they want to see
user_choice = input("Enter the class type to see (Normal, 'No Lung Opacity/Not Normal', or Lung Opacity): ")

# Ensure the user inputs a valid choice
while user_choice not in image_paths:
    print("Invalid choice. Please enter one of the specified class types.")
    user_choice = input("Enter the class type to see (Normal, 'No Lung Opacity/Not Normal', or Lung Opacity): ")

# Display the selected image
img_dir = image_paths[user_choice]
display_dicom_image(img_dir, figsize)

If the code is correct you should see something _similar_ to the following output depending upon your input choice. The example shown here is for the choice "Lung Opacity".

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image06E.png)

### Example 4: Select Image Number

The code in the cell below allows the user to select **how many images** that were downloaded should be used in the analysis. The `pna_data` file has a total of `9937` images that are `1024x1024x3` pixels in size. Let's calculate how much System RAM would be necessary to store all `9937` of these DICOM images in memory:

To determine the size of a data file that contains 9937 images, each with dimensions 1024x1024 and using 3 color channels (RGB), we can follow these steps:

1. **Calculate the size of one image:**
   - Each pixel is represented by 3 bytes (one for each color channel).
   - The image resolution is 1024 x 1024 pixels.
   - Therefore, the size of one image in bytes is:
     $
     1024 \times 1024 \times 3 = 3,145,728 \text{ bytes}
     $

2. **Calculate the total size for all images:**
   - There are 9937 such images in the data file.
   - Therefore, the total size of the data file in bytes is:
     $
     9937 \times 3,145,728 = 31,260,740,616 \text{ bytes}
     $

To put this into perspective, we can convert it to more human-readable units:

- **In kilobytes (KB):**
  $
  31,260,740,616 \div 1024 = 30,528,067 \text{ KB}
  $

- **In megabytes (MB):**
  $
  30,528,067 \div 1024 = 30,007.9 \text{ MB}
  $

- **In gigabytes (GB):**
  $
  30,007.9 \div 1024 = 29.8 \text{ GB}
  $

Therefore, the size of the data file would be approximately **29.8 gigabytes** if it contained 9937 images, each with dimensions `1024x1024` and using 3 color channels. That would require a large amount of system memory just to store the image data **BEFORE** it can be transferred to the GPU. Even with Colab's `High-RAM` option enabled, it is pretty easy to generate an **Out Of Memory (OOM)** error and crash the notebook!


In [ ]:
# Example 4: Select Image Number

MAX_FILES=-1  # -1 means all image files
SET_SEED=586

# generate image_df
print(f"Generating DICOM images with seed={SET_SEED}")
img_df, dropped = load_and_preprocess_data(
    data_dir='./pna_data',
    max_files=MAX_FILES,         # 10_000,
    seed=SET_SEED,
    log_dropped=True
)
display(img_df)

If the code is correct you should see something _similar_ to the following output depending upon your input choice.

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image07E.png)

### Example 5: Generate File Paths for Data Loader

The code in the cell below generates two very important Python `lists`.
As will be explained below, these file paths will tell the special data loader where to find the X-values (images) and y-values (Targets).


In [ ]:
# Example 5: Generate File Paths for Data Loader

# Build the two lists that the rest of your pipeline expects ----
file_paths_list = img_df['file_path'].tolist()
labels_list     = img_df['Target'].astype(int).tolist()

# --- Show first 3 items in lists -------------------------------
for i in range(3):
    print(f"X-value {i}: {file_paths_list[i]}")
    print(f"y-value {i}:     {labels_list[i]}\n")

If the code is correct you should see something _similar_ to the following output depending upon your input choice.

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image08E.png)

At first this output might appear rather odd, especially the `X-value`. Instead of actual DICOM images, the X-values are the _path names_ to each image. This is quite different than in your previous lessons where the X-values were numerical values in a `Numpy array`. The reason for generating path names and not a numerical array has to do with saving system RAM, as will be explained below.

------------------------------------------------

### **_Lazy Loading_: The Secret of Analyzing Large Image Datafiles**

In this lesson we didn't generate a **huge NumPy array** to hold the image data; instead, we are using **filenames** and **_lazy loading_** techniques to feed the data loader. This approach significantly reduces memory usage because it avoids loading entire image arrays into `System RAM` at once. Instead, images are loaded on-the-fly as needed by the training process, which can be **much more memory efficient** for large datasets.

Here's a breakdown of why _lazy loading_ is superior to generating an Numpy array in system RAM:

* **Memory Efficiency:** By using filenames and a data loader that reads images on-the-fly, the code avoids consuming nearly 30 GB of RAM all at once. This is particularly important when dealing with large datasets or high-resolution images, where each image can consume substantial memory. Lazy loading defers the loading of image data until it's actually needed for processing, thus reducing peak memory usage.

* **Scalability:** This approach scales better with larger datasets because you don't need to allocate a massive amount of RAM upfront. You only load as much data into memory as your application needs at any given time, which is ideal for scenarios where the dataset size far exceeds available system RAM.

* **Efficiency:** Lazy loading can also improve processing efficiency by reducing I/O bottlenecks. When training a model, you often need to read and preprocess images multiple times (e.g., during each epoch of training). This repeated reading from disk is more efficiently managed when the data isn't fully loaded into memory at once.

* **Simplicity:** The code can be simpler and cleaner because it doesn't involve managing large in-memory arrays. Instead, it relies on existing libraries for efficient file handling and data loading, which often have built-in optimizations for performance.

In summary, the **lazy loading technique is superior** to loading all images into a `NumPy` array due to its memory efficiency, scalability, improved processing efficiency, and simplicity. This approach allows you to work with large datasets that would otherwise be impractical to handle using traditional methods that require massive amounts of RAM.




-------------------------------------------------

### Create Data Functions



The code in the cell below creates one **class** and several **functions** that will perform a variety of tasks in preparation for train our `ResNet101` neural network.

Here is a brief summary of the activities performed by the most important functions.

------------------------------------------------
* class: **DicomImageDataset(Dataset)**
This code defines a custom dataset class named **DicomImageDataset** for handling DICOM image files. The class is designed to read and process these images along with their corresponding labels. Here's a brief synopsis of its functionality:

* * **Initialization (__init__):** The constructor takes three parameters: file_paths, labels, and an optional transform. It asserts that the lengths of file_paths and labels are equal, as each file path must have a corresponding label. The class initializes these inputs into instance variables for later use.

* * **Length (__len__):** This method returns the total number of items in the dataset, which is determined by the length of file_paths. It represents how many images are available in the dataset.

* * **Get Item (__getitem__):** This method retrieves an item from the dataset at a specified index (idx). It first extracts the file path and label.

--------------------------------------------

* def function: **build_loaders( )**
This function is designed to create train/validation DataLoaders from a dataset of DICOM images. It takes several parameters including file paths and labels, as well as optional settings for batch size, validation split ratio, number of worker threads, and whether memory should be pinned during data loading. The function returns two DataLoader objects: one for the training set and one for the validation set.

* * **Parameters:**
    - `file_paths`: List of absolute paths to the DICOM files.
    - `labels`: Corresponding integer labels for each file path.
    - `batch_size` (default: BATCH_SIZE): Number of samples per batch in the DataLoader.
    - `val_split` (default: VAL_SPLIT): Proportion of the dataset to include in the validation set.
    - `num_workers` (default: 0): Number of subprocesses to use for data loading.
    - `pin_memory` (default: False): Whether to copy tensors into CUDA pinned memory before returning them.

* * **Transforms:**
    - Two transformations are obtained based on whether the dataset is for training or validation using the `get_transform` function. The training transformation includes data augmentation, while the validation transformation does not.

* * **Dataset Splitting:**
    - A full dataset object is created from the provided file paths and labels with the training transform.
    - The size of the train and validation sets is calculated based on the `val_split` parameter.
    - The dataset is split into a training set (`train_ds`) and a validation set (`val_ds`) using the `random_split` function.

* * **Validation Transform Override:**
    - The transform for the validation set (`val_ds`) is overridden to use the validation transformation obtained earlier.

* * **DataLoaders:**
    - Train and validation DataLoader objects are created with the specified batch size, shuffling, number of worker threads, and pinned memory settings.

* * **Returns:**
    - A tuple containing two DataLoader objects: `train_loader` for the training set and `val_loader` for the validation set.
--------------------------------------------

* def function: **get_resnet101( )**

This function creates and returns a ResNet-101 model tailored for classification tasks, optionally with additional regularization layers such as dropout or DropBlock, followed by a final linear layer. The model can be initialized with pre-trained weights on ImageNet.

* * **Parameters:**
    - `num_classes`: Number of output classes the model should predict.
    - `pretrained` (default=True): If True, initializes the model with pre-trained weights from ImageNet.
    - `device` (default=None): Specifies the device (CPU or GPU) where the model will be loaded and run. If None, it automatically selects based on availability.
    - `name` (default=None): Optional human-friendly name for the model, which is stored as `model.name`.
    - `dropout_p` (default=0.5): Dropout probability to apply after the backbone layers.
    - `dropblock_size` (default=5): Size of the spatial block to drop using DropBlock regularization. This parameter is only used if `dropblock_prob` is greater than 0.
    - `dropblock_prob` (default=0.1): Probability that a block is dropped during training with DropBlock regularization.
    - `add_norm` (default=False): If True, adds a BatchNorm layer after the backbone layers.

* * **Device Handling:**
    - If no device is specified, the function will automatically select either CPU or GPU based on whether CUDA is available.

* * **Model Initialization:**
    - The model starts by loading the ResNet-101 architecture, optionally using pre-trained weights from ImageNet.
    - DropBlock regularization is applied to the final layer of the backbone if `dropblock_size` and `dropblock_prob` are provided.
    - The fully connected (fc) layer of the original ResNet-101 is replaced with an identity mapping since we will add our own classification head.

* * **Model Tail:**
    - Additional layers such as BatchNorm and Dropout can be added based on the parameters provided.
    - A new linear layer is appended to serve as the final classification layer for `num_classes`.
    - The entire sequence of layers is moved to the specified device and returned as a single sequential model.
    - If a name is provided, it is assigned to the model's `name` attribute for identification purposes.

* * **Returns:**
    - A PyTorch nn.Module object representing the ResNet-101 backbone with optional

--------------------------------------------

* def function: **train_one_epoch( )**

This function is designed to train a neural network model for one complete pass through the training dataset. It takes several parameters including the model itself, a DataLoader object for the training data, a loss criterion (e.g., CrossEntropyLoss), an optimizer (e.g., Adam or SGD), a device (CPU or GPU) on which to perform computations, and optionally a `model_name` for bookkeeping purposes. The function returns the average loss per sample over the epoch.

* * **Parameters:**
    - `model`: Neural network model to be trained.
    - `loader`: DataLoader object containing the training data.
    - `criterion`: Loss criterion (e.g., CrossEntropyLoss) for computing the loss between predictions and targets.
    - `optimizer`: Optimizer (e.g., Adam or SGD) used to update model parameters.
    - `device`: Device (CPU or GPU) on which to perform computations.
    - `model_name` (optional): A string to be stored as the model's name attribute for bookkeeping purposes.

* * **Bookkeeping:**
    - If a `model_name` is provided, it is assigned to `model.name`. This allows tracking and identification of the model during training or later inspection.

* * **Training Process:**
    - The model is set to train mode using `model.train()`.
    - A variable `epoch_loss` is initialized to accumulate the total loss over the epoch.
    - Iterating through the DataLoader, for each batch of images (`imgs`) and targets (ground truth labels), the following steps are performed:
        - Move both `imgs` and `targets` to the specified device (CPU or GPU).
        - Zero out the gradients from the previous step using `optimizer.zero_grad()`.
        - Forward pass through the model to get predictions (`outputs`).
        - Compute the loss between `outputs` and `targets` using the provided criterion.
        - Backward pass to compute gradients of the loss with respect to model parameters.
        - Update model parameters using the optimizer's step function.
        - Accumulate the batch loss to `epoch_loss`.
    - After all batches are processed, calculate and return the average epoch loss per sample (`epoch_loss / len(loader.dataset)`).
--------------------------------------------

* def function: **run_training_lazy( )**

This function orchestrates the training of a neural network model using a lazy loading approach. It handles data loading, model creation, loss computation, optimization, learning rate scheduling, and early stopping. The function is designed to be flexible with various hyperparameters and can optionally use GPU acceleration if available.

* * **Parameters:**
    - `file_paths`: List of file paths to the training images.
    - `labels`: Corresponding list of integer labels for each image.
    - `num_epochs` (default: NUM_EPOCHS): Number of epochs to train the model.
    - `batch_size` (default: BATCH_SIZE): Number of samples per batch in the DataLoader.
    - `lr` (default: LEARNING_RATE): Learning rate for the optimizer.
    - `weight_decay` (default: WEIGHT_DECAY): L2 regularization strength.
    - `patience` (default: PATIENCE): Number of epochs with no improvement after which training will be stopped.
    - `val_split` (default: VAL_SPLIT): Fraction of the dataset to use as validation set.
    - `early_stop` (default: EARLY_STOPPING): Boolean flag indicating whether to stop early if validation loss does not improve.
    - `device` (default: None): Device (CPU or GPU) where the model will be loaded and run. If None, it automatically selects based on availability.

* * **Device Handling:**
    - If no device is specified, the function will automatically select either CPU or GPU based on whether CUDA is available.

* * **Data Loaders:**
    - `train_loader` and `val_loader` are created using the `build_loaders` function with parameters for batch size, validation split, number of worker threads, and pinned memory.

* * **Model Initialization:**
    - A ResNet-101 model is instantiated with specified hyperparameters such as dropout probabilities, normalization addition, and a custom name. The number of classes (`num_classes`) is set based on the maximum label value plus one.

* * **Loss/Optimizer/Scheduler:**
    - CrossEntropyLoss is used for loss computation with optional label smoothing.
    - AdamW optimizer is initialized with specified learning rate and weight decay.
    - CosineAnnealingLR scheduler adjusts the learning rate during training.

* * **Early Stopping:**
    - A best model checkpoint is maintained to revert to if validation performance does not improve over a set number of epochs (`patience`).

* * **Training Loop:**
    - The function iterates through each epoch, performing both training and validation phases:
        - During training, it computes gradients, updates weights, and accumulates loss.
        - During validation, it evaluates the model without updating parameters, accumulating loss and accuracy metrics.
    - Learning rate is adjusted using the scheduler at the end of each epoch.
    - Early stopping is implemented to halt training if performance does not improve for a specified number of epochs.

* * **Final Report:**
    - After training, the best performing model (based on validation loss) is reported along with its metrics.
    - Memory cache is cleared and garbage collected at the end of training to free up resources.

* * **Returns:**
    - A dictionary containing the training and validation losses, accuracies, as well as the best epoch and associated metrics.
--------------------------------------------

* def function: **plot_train_hist( )**

This function generates a two-panel plot to visualize the training and validation performance metrics over epochs. The left panel displays accuracy trends during training and validation, while the right panel shows loss trends. Optionally, it highlights the epoch with the best model performance using a vertical red dashed line.

* * **Parameters:**
    - `history`: A dictionary containing keys for 'train_acc', 'val_acc', 'train_loss', and 'val_loss'. These keys should map to lists or arrays of metrics recorded over each epoch during training.

* * **Plot Layout:**
    - The plot consists of two subplots arranged horizontally: one on the left for accuracy, and one on the right for loss. Each subplot is configured with appropriate labels, titles, and limits as follows:
        - Left panel (accuracy):
            - Plots training and validation accuracy curves.
            - Sets x-axis label to "Epoch", y-axis label to "Accuracy", and ensures the y-axis range is between 0 and 1 for better interpretability of the metrics.
            - The legend indicates which curve corresponds to train and validation accuracy, positioned at the bottom right.
        - Right panel (loss):
            - Plots training and validation loss curves.
            - Sets x-axis label to "Epoch", y-axis label to "Loss".
            - Ensures the y-axis starts from 0 for better visual differentiation of trends in the loss values. The top limit is left to matplotlib's default setting, which generally adjusts it based on the data range.
            - The legend indicates which curve corresponds to train and validation loss, positioned at the top right.

* * **Highlight Best Epoch:**
    - If a 'best_epoch' key exists in the `history` dictionary, indicating that early stopping was applied during training, a vertical red dashed line is drawn at this epoch on both subplots to highlight it as the optimal point where performance started improving significantly.
    - The legend for each axis is updated dynamically to include the "Best epoch" label if not already present.

* * **Figure Title and Layout:**
    - A main title "Training History" is added at the top of the figure, ensuring it does not overlap with subplots.
    - `fig.tight_layout(rect=[0, 0, 1, 0.95])` adjusts the layout to ensure all elements fit well without overlapping, leaving some space for the title.

* * **Display the Plot:**
    - The plot is displayed using `plt.show()`, which opens a window showing both subplots side by side.

In [ ]:
# Create Data Functions

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from torchvision import transforms, models
from tqdm import tqdm
import torch.nn.functional as F
import copy
from typing import Optional

# Set Import Variables Here!
NUM_EPOCHS: int = 50
BATCH_SIZE: int = 64  # Reduced for memory management
IMG_SIZE: int = 512  # Standard size for ResNet101
LEARNING_RATE: float = 0.0010
WEIGHT_DECAY: float  = 0.0010
VAL_SPLIT: float = 0.2
PATIENCE = 5
EARLY_STOPPING = True

# ------------------------------------------
# Custom Dataset Class with Transforms
# ------------------------------------------
class DicomImageDataset(Dataset):
    """
    Load a DICOM image from disk and return (image_tensor, label).

    Parameters
    ----------
    file_paths : list[str]
        Absolute paths to the DICOM files.
    labels    : list[int]
        Integer labels that align with `file_paths`.
    transform : callable | None
        Optional transform that receives the `pydicom.dataset.FileDataset`
        and returns a torch tensor.
    """

    def __init__(self, file_paths, labels, transform=None):
        assert len(file_paths) == len(labels), \
            f"file_paths ({len(file_paths)}) != labels ({len(labels)})"
        self.file_paths = file_paths
        self.labels    = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path  = self.file_paths[idx]
        label = self.labels[idx]

        try:
            dicom_img = pydicom.dcmread(path, force=True)   # <-- fails on corrupt files
        except Exception as exc:
            # Raise a RuntimeError that will be caught by the caller.
            raise RuntimeError(f"Could not read DICOM at {path!r}: {exc}")

        # Convert DICOM pixel array to proper format for transforms
        if hasattr(dicom_img, 'pixel_array'):
            img_array = dicom_img.pixel_array

            # Handle different data types and normalize
            if img_array.dtype != np.uint8:
                if img_array.max() > 1.0:
                    img_array = ((img_array - img_array.min()) /
                               (img_array.max() - img_array.min()) * 255).astype(np.uint8)
                else:
                    img_array = (img_array * 255).astype(np.uint8)

            # Convert to PIL Image for transforms
            from PIL import Image
            if len(img_array.shape) == 2:
                # Single channel - convert to RGB by duplicating
                pil_img = Image.fromarray(img_array, mode='L')
                # Convert grayscale to RGB
                pil_img = pil_img.convert('RGB')
            else:
                # Already multi-channel, convert to RGB if needed
                pil_img = Image.fromarray(img_array)
                if pil_img.mode != 'RGB':
                    pil_img = pil_img.convert('RGB')

            if self.transform is not None:
                img = self.transform(pil_img)
            else:
                # Convert to tensor if no transform
                img = torch.tensor(img_array, dtype=torch.float32)
                if len(img.shape) == 2:
                    img = img.unsqueeze(0)  # Add channel dimension
        else:
            raise RuntimeError(f"No pixel array in DICOM file {path!r}")

        return img, label


# ------------------------------------------
# Helper: Check file paths
# ------------------------------------------
def filter_valid_dicom(paths, labels, max_attempts=5):
    """
    Returns a pair (valid_paths, valid_labels) that only contains files that
    can be successfully read with pydicom.
    """
    valid_paths, valid_labels = [], []
    for p, l in zip(paths, labels):
        try:
            pydicom.dcmread(p, force=True)
        except Exception as exc:
            # Log and skip
            print(f"Skipping {p!r}: {exc}")
            continue
        valid_paths.append(p)
        valid_labels.append(l)

    if not valid_paths:
        raise RuntimeError("All DICOM files failed to load – dataset is empty.")
    return valid_paths, valid_labels

# ------------------------------------------
# Helper: Build transforms
# ------------------------------------------
def get_transform(
    img_size=IMG_SIZE,
    is_train: bool = True,
    crop_size=IMG_SIZE,
    h_flip: bool = True,
    augment: bool = False
) -> transforms.Compose:
    """
    Returns a torchvision transform chain.
    """
    if is_train:
        transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomResizedCrop(crop_size) if augment else transforms.CenterCrop(crop_size),
            transforms.RandomHorizontalFlip() if h_flip else transforms.Lambda(lambda x: x),
            transforms.ToTensor(),  # This will convert to float and normalize [0,1]
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
    else:  # eval / test
        transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.CenterCrop(crop_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])

    return transform

# ------------------------------------------------------------------
# Helper: Build loaders from lazy datasets
# ------------------------------------------------------------------
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import ResNet101_Weights
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import gc

def build_loaders(
    file_paths, labels,
    batch_size: int = BATCH_SIZE,
    val_split:  float = VAL_SPLIT,
    num_workers: int = 0,
    pin_memory: bool = False
):
    """Create train/validation DataLoaders from lazy dataset."""
    transform_train = get_transform(is_train=True)
    transform_val   = get_transform(is_train=False)

    full_dataset = DicomImageDataset(file_paths, labels, transform=transform_train)

    train_size = int(len(full_dataset) * (1.0 - val_split))
    val_size   = len(full_dataset) - train_size
    train_ds, val_ds = random_split(full_dataset, [train_size, val_size])

    # Override validation transform
    val_ds.dataset.transform = transform_val

    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=pin_memory
    )
    val_loader = DataLoader(
        val_ds, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=pin_memory
    )
    return train_loader, val_loader

# ------------------------------------------
# Training loop
# ------------------------------------------
from tqdm import tqdm

def train_one_epoch(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
    model_name: str | None = None,
) -> float:
    """
    Train a single epoch.  The new ``model_name`` argument is *only* for
    bookkeeping – it is stored on the model as ``model.name``.
    """
    if model_name is not None:
        # Store the name for later inspection
        model.name = model_name

    model.train()
    epoch_loss = 0.0

    for imgs, targets in tqdm(loader, desc="Training", leave=False):
        imgs, targets = imgs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * imgs.size(0)

    return epoch_loss / len(loader.dataset)


# --------------------------------------------
# Measure validation loss during training
# --------------------------------------------
def validate(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    device: torch.device,
) -> tuple[float, float]:
    model.eval()
    epoch_loss = 0.0
    correct = 0

    with torch.no_grad():
        for imgs, targets in tqdm(loader, desc="Validation", leave=False):
            imgs, targets = imgs.to(device), targets.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, targets)

            epoch_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == targets).sum().item()

    val_loss = epoch_loss / len(loader.dataset)
    val_acc = correct / len(loader.dataset)
    return val_loss, val_acc

import torch
import torch.nn as nn
from torchvision import models
from dropblock import DropBlock2D  # pip install dropblock

# --------------------------------------------
# Create ResNet model
# --------------------------------------------
def get_resnet101(
    num_classes: int,
    pretrained: bool = True,
    device: torch.device | None = None,
    name: str | None = None,
    dropout_p: float = 0.5,
    dropblock_size: int | None = 5,
    dropblock_prob: float | None = 0.1,
    add_norm: bool = False,
    **kwargs,
) -> nn.Module:
    """
    Return a ResNet‑101 backbone followed by optional regularisers
    and a final linear head.

    Parameters
    ----------
    num_classes : int
        Number of output classes.
    pretrained : bool, default=True
        Load ImageNet pre‑trained weights.
    device : torch.device | None, default=None
        Device to move the model onto.
    name : str | None, default=None
        Optional human‑friendly name (`model.name = name`).
    dropout_p : float, default=0.5
        Drop‑out probability *after* the backbone.
    dropblock_size : int | None, default=5
        Size of the spatial block to drop (used only if dropblock_prob > 0).
    dropblock_prob : float | None, default=0.1
        Probability that a block is dropped.
    add_norm : bool, default=False
        Add a 1‑D BatchNorm layer *after* the backbone.

    Returns
    -------
    nn.Module
        Sequential model: `backbone → (optional layers) → Linear`
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the pre‑trained ResNet‑101
    backbone = models.resnet101(
        weights=models.ResNet101_Weights.DEFAULT if pretrained else None
    )

    # Insert DropBlock *before* the global avg‑pool
    if dropblock_size is not None and dropblock_prob is not None:
        # The final residual block produces a 4‑D tensor (b, 2048, 7, 7)
        backbone.layer4 = nn.Sequential(
            backbone.layer4,
            DropBlock2D(block_size=dropblock_size, drop_prob=dropblock_prob)
        )

    # Make the backbone output a 1‑D vector (2048‑dim)
    in_features = backbone.fc.in_features
    backbone.fc = nn.Identity()

    # Build the tail
    layers: list[nn.Module] = [backbone]

    if add_norm:
        layers.append(nn.BatchNorm1d(in_features))

    if dropout_p > 0.0:
        layers.append(nn.Dropout(p=dropout_p))

    # Final classification head
    layers.append(nn.Linear(in_features, num_classes))

    # Stack everything
    model = nn.Sequential(*layers).to(device)

    if name is not None:
        model.name = name

    return model


# ------------------------------------------
# Training routine
# ------------------------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision import models
import copy, gc

def run_training_lazy(
    file_paths: list[str],
    labels:     list[int],
    num_epochs: int   = NUM_EPOCHS,
    batch_size: int   = BATCH_SIZE,
    lr:          float = LEARNING_RATE,
    weight_decay:float = WEIGHT_DECAY,
    patience:    int   = PATIENCE,
    val_split:   float = VAL_SPLIT,
    early_stop:  bool  = EARLY_STOPPING,
    device:      torch.device = None
):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ----- Loaders
    train_loader, val_loader = build_loaders(
        file_paths, labels,
        batch_size=batch_size,
        val_split=val_split,
        num_workers=0,
        pin_memory=False
    )

    # ----- Model
    num_classes = max(labels) + 1
    # model = get_resnet101(num_classes=num_classes, pretrained=True, device=device)
    model = get_resnet101(
    num_classes=10,
    pretrained=True,
    dropout_p=0.5,
    dropblock_size=5,
    dropblock_prob=0.1,
    add_norm=True,
    name="resnet101_reg"
)

    # ----- Loss/optimiser/scheduler
    #criterion = nn.CrossEntropyLoss()
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    #optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs, eta_min=lr * 0.01
    )

    # ----- Early‑stopping bookkeeping
    best_val_loss = float("inf")
    best_epoch = 0
    best_train_acc = best_val_acc = None
    best_model_wts = copy.deepcopy(model.state_dict())
    epochs_without_improve = 0

    # ----- History
    train_losses, val_losses = [], []
    train_accs,   val_accs   = [], []

    # ----- Epoch loop
    for epoch in range(1, num_epochs + 1):
        # training
        model.train()
        epoch_train_loss = 0.0
        correct_train = 0
        n_train = len(train_loader.dataset)

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss.item() * xb.size(0)
            preds = torch.argmax(logits, dim=1)
            correct_train += (preds == yb).sum().item()

        epoch_train_loss /= n_train
        epoch_train_acc  = correct_train / n_train

        # validation
        model.eval()
        epoch_val_loss = 0.0
        correct_val = 0
        n_val = len(val_loader.dataset)

        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                loss = criterion(logits, yb)

                epoch_val_loss += loss.item() * xb.size(0)
                preds = torch.argmax(logits, dim=1)
                correct_val += (preds == yb).sum().item()

        epoch_val_loss /= n_val
        epoch_val_acc  = correct_val / n_val

        # scheduler step
        scheduler.step()

        # early stopping
        if early_stop:
            if epoch_val_loss < best_val_loss - 1e-5:
                best_val_loss = epoch_val_loss
                best_epoch = epoch
                best_train_acc = epoch_train_acc
                best_val_acc   = epoch_val_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                epochs_without_improve = 0
            else:
                epochs_without_improve += 1

            if epochs_without_improve >= patience:
                print("\nEarly stopping triggered.")
                print(f"Best epoch (before stopping): {best_epoch}")
                print(f"  Train Acc: {best_train_acc:.4f} | Train Loss: {epoch_train_loss:.4f}")
                print(f"  Valid Acc: {best_val_acc:.4f} | Valid Loss: {epoch_val_loss:.4f}\n")
                model.load_state_dict(best_model_wts)
                break

        # log
        print(
            f"Epoch {epoch}/{num_epochs} | "
            f"Train Acc:  {epoch_train_acc:.4f} | Train Loss: {epoch_train_loss:.4f} | "
            f"Val Acc:    {epoch_val_acc:.4f} | Val Loss: {epoch_val_loss:.4f}"
        )

        # history
        train_losses.append(epoch_train_loss)
        train_accs.append(epoch_train_acc)
        val_losses.append(epoch_val_loss)
        val_accs.append(epoch_val_acc)

    # final report
    if not early_stop or epochs_without_improve < patience:
        best_epoch = epoch
        best_train_acc = epoch_train_acc
        best_val_acc   = epoch_val_acc
        best_val_loss  = epoch_val_loss

    print("\nTraining finished.")
    print(f"Best epoch: {best_epoch}")
    print(f"  Train Acc: {best_train_acc:.4f} | Train Loss: {epoch_train_loss:.4f}")
    print(f"  Val   Acc: {best_val_acc:.4f}   | Val   Loss: {epoch_val_loss:.4f}")

    torch.cuda.empty_cache()
    gc.collect()

    return {
        "train_loss": train_losses,
        "train_acc":   train_accs,
        "val_loss":    val_losses,
        "val_acc":     val_accs,
        "best_epoch":  best_epoch,
        "best_train_acc": best_train_acc,
        "best_val_acc":   best_val_acc,
        "best_train_loss": epoch_train_loss,
        "best_val_loss":   epoch_val_loss,
    }


# -------------------------------------------------------------
#  Plot training history
# -------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt

def plot_train_hist(history: dict) -> None:
    """
    Plot a professional two-panel figure:
        • Left panel – train & val accuracy.
        • Right panel – train & val loss.

    Parameters
    ----------
    history : dict
        Must contain the keys 'train_acc', 'val_acc',
        'train_loss', 'val_loss'.
        Optional key: 'best_epoch' (1-based index).
    """
    # Use a clean style
    plt.style.use('seaborn-v0_8-whitegrid')

    num_epochs = len(history["train_acc"])
    epochs = np.arange(1, num_epochs + 1)

    # ---- 2-panel layout ---------------------------------------
    fig, (ax_acc, ax_loss) = plt.subplots(1, 2, figsize=(12, 6))

    # ==========================================
    # LEFT PANEL: ACCURACY
    # ==========================================
    ax_acc.plot(epochs, history["train_acc"], label="Training Accuracy",
                color="#1f77b4", marker='o', markersize=4, linewidth=2)
    ax_acc.plot(epochs, history["val_acc"],   label="Validation Accuracy",
                color="#ff7f0e", marker='s', markersize=4, linewidth=2)

    ax_acc.set_xlabel("Epoch", fontsize=12, fontweight='bold')
    ax_acc.set_ylabel("Accuracy", fontsize=12, fontweight='bold')
    ax_acc.set_ylim(0, 1.05)
    ax_acc.set_title("Model Accuracy", fontsize=14)
    ax_acc.legend(loc="lower right", frameon=True)
    ax_acc.grid(True, linestyle='--', alpha=0.7)

    # ==========================================
    # RIGHT PANEL: LOSS
    # ==========================================
    ax_loss.plot(epochs, history["train_loss"], label="Training Loss",
                 color="#2ca02c", marker='o', markersize=4, linewidth=2)
    ax_loss.plot(epochs, history["val_loss"],   label="Validation Loss",
                 color="#d62728", marker='s', markersize=4, linewidth=2)

    ax_loss.set_xlabel("Epoch", fontsize=12, fontweight='bold')
    ax_loss.set_ylabel("Loss", fontsize=12, fontweight='bold')
    ax_loss.set_ylim(bottom=0)
    ax_loss.set_title("Model Loss", fontsize=14)
    ax_loss.legend(loc="upper right", frameon=True)
    ax_loss.grid(True, linestyle='--', alpha=0.7)

    # ==========================================
    # HIGHLIGHT BEST EPOCH
    # ==========================================
    if "best_epoch" in history:
        best_ep = history["best_epoch"]
        best_idx = best_ep - 1 # Convert 1-based epoch to 0-based index

        # 1. Vertical Line on both plots
        for ax in (ax_acc, ax_loss):
            ax.axvline(best_ep, color="gray", linestyle="--", linewidth=1.5, alpha=0.8)
            # Add text label at the top of the line
            ax.text(best_ep, ax.get_ylim()[1]*0.95, f' Best Epoch: {best_ep}',
                    color="gray", fontweight='bold', ha='left')

        # 2. Annotate value on Accuracy Plot
        best_val_acc = history["val_acc"][best_idx]
        ax_acc.plot(best_ep, best_val_acc, marker='*', color='gold', markersize=15, markeredgecolor='black')
        ax_acc.annotate(f"{best_val_acc:.2f}",
                        (best_ep, best_val_acc),
                        textcoords="offset points", xytext=(-20, 10), ha='center',
                        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", alpha=0.8))

        # 3. Annotate value on Loss Plot
        best_val_loss = history["val_loss"][best_idx]
        ax_loss.plot(best_ep, best_val_loss, marker='*', color='gold', markersize=15, markeredgecolor='black')
        ax_loss.annotate(f"{best_val_loss:.2f}",
                        (best_ep, best_val_loss),
                        textcoords="offset points", xytext=(-20, 10), ha='center',
                        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", alpha=0.8))

    # Main Title
    fig.suptitle("Training History Metrics", fontsize=18, fontweight='bold', y=0.98)
    fig.tight_layout(rect=[0, 0, 1, 0.95])

    plt.show()

If the code is correct, you should _not_ see any output.

### Setup Device

The code in the cell below double-checks to see if there is a Nvidia GPU available to PyTorch to train the neural network.  The code then issues a warning if no GPU is available.

In [ ]:
# Setup device

import torch

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device("cuda"):
    print(f"Using device: {device} for neural network training")
elif device == torch.device("cpu"):
   print(f"WARNING: While this code can run on a CPU, it will take many hours to run a single epoch!")
else:
  print("No device found!")

If the code is correct and you are using a GPU, you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image09E.png)


### Example 6: Run Training

This code snippet initiates the training process for a classification model, measuring and printing the total elapsed time taken to complete the training across a specified number of epochs (NUM_EPOCHS).

The key steps involved are:

* **Start Training Notification:** It prints a message indicating that the training is starting for the given number of epochs, along with other relevant details such as batch size and learning rate.

* **Time Measurement:** It captures the current time before commencing the training using time.time() to serve as the start time. This will be used later to calculate the total elapsed time during and after the training process.

* **Training Execution:** The function `run_training_lazy` is called with parameters for file paths, labels, epochs, batch size, learning rate, weight decay, patience (for early stopping), validation split fraction, and a flag indicating whether to use early stopping. This function performs the actual training of the model using these inputs. The suffix `lazy` refers to the fact that it is a "lazy" or "delayed evaluation" function. In programming, a `lazy function` is one that defers the computation or execution of its result until it is explicitly demanded, typically on request (e.g., when accessed). This approach can be particularly useful for optimizing performance by reducing unnecessary operations and improving efficiency, especially with large datasets or complex computations.

* **End Time Recording:** After the training is complete, it records the current time again using time.time() to calculate the total elapsed time by subtracting the start time from this end time.

* **Elapsed Time Output:** The code prints out the total elapsed time in a human-readable format (hours:minutes:seconds) using the function hms_string(elapsed_time) which likely converts the elapsed seconds into hours, minutes, and seconds for easier readability. This helps users understand how much time was spent on training.

The purpose of this code is to provide user feedback regarding both the progress (training epochs completed so far) and efficiency (total time taken for all epochs), thus aiding in performance monitoring and debugging if needed.

### **TIME ALERT**

You will be training a relatively large dataset. You can expect training to require **more than 1 hr**.

In [ ]:
# Run Training

import time

# ------------------------------------------------------------------------
# Start training
# ------------------------------------------------------------------------
print(f"-- Training (classification) is starting for {NUM_EPOCHS} epochs----------------------------")
start_time = time.time()

# Run training
history = run_training_lazy(
    file_paths=file_paths_list,
    labels=labels_list,
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    patience=PATIENCE,
    val_split=VAL_SPLIT,
    early_stop=EARLY_STOPPING
)

# Record end time
elapsed_time = time.time() - start_time

# Print elapsed time
print(f"Elapsed time: {hms_string(elapsed_time)}")

If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image10E.png)


### Example 7: Plot Training History

The code in the cell below generates two graphs. The graph on the left shows the change during each epoch in the model's accuracy in predicting the correct label for each training image (`train_acc`) and for each validation image (`val_acc`) while the graph on the right shows model's loss for each epoch for the training images (`train_loss`) and for the validation images (`val loss`). If the model was stopped early (`EarlyStopping`), a vertical red line indicates the `best epoch`.

It should be noted that `EarlyStopping` is best on the lowest `val_loss` value, not the highest `val_acc` value. There are a variety of reasons for doing it this way which are beyond the scope of this class lesson.

In [ ]:
# Example 7: Plot Training History
plot_train_hist(history=history)


If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co/BIO1173/images/class_06/class_06_3_image11E.png)

### **Analysis of Training History Metrics**

The graphs above visualize the performance of the machine learning model over roughly 21 epochs. The plots track two key metrics: **Accuracy** (higher is better) and **Loss** (lower is better) for both the training and validation datasets.

##### **1. Model Accuracy (Left Panel)**

* **Training Accuracy (Blue Line):** The model learns the training data extremely fast. By Epoch 6, it reaches nearly **100% accuracy (1.0)** and stays there. This indicates the model has effectively memorized the training examples.
* **Validation Accuracy (Orange Line):** The model's performance on unseen data improves initially but plateaus around **91% (0.91)**. It does not improve significantly after Epoch 7.
* **The "Best Epoch":** The star annotation highlights **Epoch 17**, which achieved the maximum validation accuracy of **0.91**.

##### **2. Model Loss (Right Panel)**

* **Training Loss (Green Line):** The loss decreases rapidly and smoothly, stabilizing around **0.5**, confirming that the model is effectively minimizing errors on the training set.
* **Validation Loss (Red Line):** This metric is much "noisier" (bumpy) than the training loss. It drops initially but hits a floor around **0.73** and stops improving. Crucially, it stays consistently higher than the training loss.

##### **3. Key Diagnosis: Overfitting**
The most distinct feature of these graphs is the large **gap** (divergence) between the training curves and the validation curves.

* **The Gap:** The model performs significantly better on training data (Accuracy ~1.0) than on validation data (Accuracy ~0.91).
* **Interpretation:** This is a classic signature of **Overfitting**. The model has learned the specific details or "noise" of the training data too well, preventing it from generalizing perfectly to new data.
* **Conclusion:** While 91% accuracy is a strong result, the flatline in the validation metrics suggests the model has reached its capacity. Training for more epochs beyond this point is unlikely to yield better results.

## **Lesson Turn-in**

When you have completed and run all of the code cells, use the **File --> Print.. --> Save to PDF** to generate a PDF of your Colab notebook if you are using a Mac. If you are using a Windows computer, then use the **File --> Print.. --> Microsoft Print to PDF** to generate a PDF of your Colab notebook. Save your PDF as `Class_06_3.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.

# **Lizard Tail**

## **Nano Banana Pro**

#### **1. Introduction: The Supercomputer in Your Pocket**

In the grand history of scientific tooling, there are moments of distinct punctuation—points where a barrier to entry completely collapses. The microscope allowed anyone to see the invisible. The polymerase chain reaction (PCR) allowed anyone to amplify the infinitesimal. Today, we are living through the next great punctuation: the democratization of high-performance artificial intelligence.

We refer to this ecosystem as the **"Nano Banana Pro."**

While this may sound whimsical, the name encapsulates a profound shift in how biology and medicine are practiced.
* **"Nano"** represents accessibility and ubiquity. It requires no massive server racks, no expensive dedicated workstations, and no complex installation processes. It runs on the "thin clients" we carry every day—our laptops, tablets, and phones.
* **"Banana"** symbolizes the organic, flexible, and user-friendly nature of the platform. Just as a banana comes in its own wrapper and is ready to consume, this tech stack is pre-packaged (via Google Colab) and ready to "peel" open and use immediately.
* **"Pro"** belies the accessibility. Beneath the simple interface lies the raw power of Google’s TPUs (Tensor Processing Units) and the Gemini multimodal foundation models—tools that rival the computing power available to top research institutions just a decade ago.

For the biology major or premed student, the Nano Banana Pro is not just a calculator; it is a limitless lab partner, a tireless research assistant, and a creative engine. This analysis explores the history, mechanics, and boundless applications of this ecosystem, serving as a roadmap for your future in the bio-digital age.

#### **2. A History of Convergence: From Command Lines to Chatbots**

To understand the power you hold in your hands with a simple API key, we must understand the "Dark Ages" that preceded it.

### The Era of "Iron and Syntax" (1990s–2010s)
For decades, computational biology (bioinformatics) was a gated community. To analyze a genome or model a protein, you needed three things that most biology undergraduates did not have:
1.  **Heavy Iron:** Physical access to a Linux cluster or a high-performance workstation.
2.  **Arcane Knowledge:** Fluency in the command line (BASH), Perl, or C++.
3.  **Patience:** Setting up software environments often took longer than the experiment itself.

Biology students were taught that "wet lab" (pipettes and petri dishes) and "dry lab" (code and servers) were separate worlds. If you wanted to cross over, you had to leave your biology intuition behind and become a computer scientist.

### The Jupyter Revolution
The first crack in the wall came with the **Jupyter Notebook**. It allowed code to be written in blocks, mixed with text and images, and run interactively. It turned coding from a "script" into a "narrative." Python became the lingua franca of science because of its readability.

### The Cloud Shift: Google Colab
In late 2017, Google released Colaboratory (Colab). It was a Jupyter notebook environment running entirely in the cloud. Crucially, Google offered free access to GPUs and TPUs. Suddenly, a student with a $200 Chromebook could train a deep learning model that previously required a $5,000 workstation. The hardware barrier was obliterated.

### The Cognitive Leap: The Transformer & Gemini
While Colab solved the *hardware* problem, the *software* problem remained: you still had to know how to write code perfectly. A missing semicolon or a misspelled variable would crash your simulation.

This changed with the advent of the **Transformer architecture** (the "T" in ChatGPT) and the rise of Large Language Models (LLMs). Google's Gemini represents the pinnacle of this evolution. Unlike earlier models that were "text-only," Gemini is **multimodal**—native in text, code, images, and video.

When we combine the compute of Colab with the cognition of Gemini, we get the **Nano Banana Pro**. The barrier of syntax is removed. You no longer need to speak "Computer" fluently; the computer now speaks "Biologist."

#### **3. The "Nano Banana Pro" in the Professional Realm**

For students aiming for medical school, graduate research, or the biotech industry, mastery of this ecosystem is a career accelerant. Here is how it applies to professional workflows.

### A. The "Pocket Pathologist": Medical Imaging
In modern medicine, the volume of diagnostic imaging (X-rays, CT scans, Histology slides) is outpacing the number of human radiologists. AI is becoming the standard "first pass" triage tool.

Using Gemini's vision capabilities within Colab, a student can prototype diagnostic tools.
* **Scenario:** You are studying hematology.
* **Workflow:** You upload an image of a blood smear. You ask the Nano Banana Pro agent to "Identify and count the basophils and eosinophils."
* **Result:** The model returns a structured count and highlights the cells.
* **Professional Skill:** Understanding how to validate these AI outputs—knowing where the AI fails and where it succeeds—is a critical skill for the doctors of tomorrow. You will likely be working alongside these "AI Residents" in your future practice.


### B. Structural Biology and Drug Discovery
Understanding how proteins fold and interact is the "Holy Grail" of pharmacology. Tools like AlphaFold have revolutionized this field.

* **Scenario:** You want to understand why a specific mutation causes Cystic Fibrosis.
* **Workflow:** You use your Colab environment to retrieve the protein structure prediction for the CFTR protein. You then ask the AI to "Highlight the Phenylalanine-508 residue and visualize the hydrophobic pocket."
* **Result:** You get a 3D interactive visualization.
* **Professional Skill:** This moves you from memorizing pathway diagrams to visualizing molecular mechanics. In the pharmaceutical industry, this "in silico" (computer simulation) testing saves billions of dollars before "in vitro" (test tube) experiments begin.


### C. Genomic Data Wrangling
Genomics produces "Big Data"—terabytes of A, C, T, and Gs. "Wrangling" this data (cleaning it, formatting it, finding patterns) is 80% of a bioinformatician's job.

* **Scenario:** You have a messy text file of patient gene sequences mixed with clinical notes.
* **Workflow:** You paste the mess into Colab and tell the agent: *"Extract the gene sequences, convert them to FASTA format, and create a JSON file linking each patient's age to their specific mutation."*
* **Result:** What used to take 4 hours of writing Regular Expressions (RegEx) now takes 30 seconds.
* **Professional Skill:** Speed. In a clinical research setting, the ability to clean data instantly makes you the most valuable person on the team.


![__](https://biologicslab.co/BIO1173/images/class_06/DNA_image.png)


#### **4. The "Nano Banana Pro" for Personal Productivity**

You don't need to be curing cancer to use these tools. The Nano Banana Pro is also a massive upgrade to your personal study habits and daily life as a student.

### A. The Socratic Study Buddy
Biology requires memorizing massive amounts of information. Static flashcards are boring and lack context.

* **Use Case:** You are studying for a Biochemistry final on the Krebs Cycle.
* **Action:** You initialize a "Tutor Agent" in Colab with a system instruction: *"You represent the enzyme Citrate Synthase. Quiz me on my inputs and outputs. If I get it wrong, give me a hint related to molecular energy."*
* **Benefit:** This creates an active recall loop that is personalized to your weak points. It turns studying into a conversation rather than a chore.

![__](https://biologicslab.co/BIO1173/images/class_06/TCA.jpg)


### B. The "De-Jargonizer"
Academic papers are notoriously difficult to read. They are dense, filled with acronyms, and written in passive voice.

* **Use Case:** You need to read five papers on "Epigenetic Methylation" for a seminar tomorrow.
* **Action:** You paste the abstract and results sections into your Colab notebook. You prompt the AI: *"Explain these findings using an analogy involving a library and locked books. Summarize the statistical significance of their results."*
* **Benefit:** The AI translates "high-science" into concepts you already understand, allowing you to grasp the *logic* of the paper before you dive into the technical details.

### C. Coding for "Life Hacks"
Because the Nano Banana Pro can write Python code for you, you can automate mundane tasks even if you aren't a programmer.

* **Use Case:** You have a spreadsheet of your lab hours or grades that you need to visualize to track your progress.
* **Action:** You upload the CSV file and ask: *"Plot my cumulative GPA over the last 4 semesters and project what grades I need to get a 3.8 average."*
* **Benefit:** You gain data literacy and actionable insights into your own life without needing to master Excel macros.


#### **5. Strengths of the Ecosystem: Why This Matters**

Why choose this specific "Nano Banana Pro" stack (Colab + Gemini) over just using ChatGPT or a standard textbook?

### Strength 1: Reproducibility and Documentation
In science, if you didn't write it down, it didn't happen. When you work in a Colab notebook, you are creating a **living document**. You have your code, your AI prompts, the AI's output, and your own notes all in one file. This is the gold standard for scientific reproducibility. You can hand your notebook to a peer, and they can run it and get the exact same results.

### Strength 2: Multimodality is Biological Reality
Biology is not text. Biology is visual (histology, anatomy), spatial (protein folding), and numerical (population statistics). A text-only AI is blind to half of biology. The Gemini model's ability to "see" images and graphs makes it uniquely suited for the life sciences.



### Strength 3: The "Sandbox" Safety
In a wet lab, if you make a mistake, you might waste expensive reagents or create a hazardous spill. In the Nano Banana Pro ecosystem, the cost of failure is zero. You can crash the code, delete the data, or hallucinate a simulation—and fix it with a simple "Restart Runtime." This psychological safety encourages play, and play is the highest form of research.


#### **6. Ethical Considerations for the Premed Student**

With great power comes great responsibility. Using AI in biology requires a new set of ethics.

1.  **Patient Privacy (HIPAA):** Never, ever put real patient names or identifying data into a public cloud model. In our exercises, we use synthetic (fake) data. In your future career, you must be strictly aware of data privacy laws.
2.  **Hallucination Check:** LLMs can "hallucinate"—they can confidently state biological facts that are wrong. You must always act as the **"Human in the Loop."** Never trust the AI blindly for clinical decisions. Use it to generate ideas, but verify them with primary literature.
3.  **Intellectual Integrity:** Using AI to write your lab report essay is plagiarism. Using AI to analyze your data and generating code to visualize it is **computational science**. Learn the difference.

---

#### **7. The Future: Agentic Biology**

Where is the Nano Banana Pro going next? We are moving from "Chatbots" to "Agents."

Currently, you ask a question, and the AI answers. In the near future, you will give a **goal**, and the AI will execute a series of steps to achieve it.
* **Current:** "Write code to plot this data."
* **Future:** "Analyze this dataset, find the outliers, search PubMed for potential reasons why these outliers exist, and draft a preliminary hypothesis."

We are entering an era of **Agentic Biology**, where software agents will run autonomous experiments in the cloud (or even drive robotic wet labs) while you direct the overall strategy. Learning to prompt and guide these agents today is the best preparation for the scientific leadership roles of tomorrow.


### **Conclusion: Peel the Banana**

The "Nano Banana Pro" is not a product you buy; it is a mindset you adopt. It is the realization that the barrier between "thinking about an experiment" and "running an experiment" has dissolved.

For the BIO 1173 student, this ecosystem offers a superpower. It allows you to punch above your weight class. You can analyze data like a PhD student, visualize concepts like a professional illustrator, and build tools like a software engineer—all while sitting in your dorm room.

The history of biology is the history of tools that extend human perception. The microscope extended our sight. The Nano Banana Pro extends our cognition. The tool is ready, powerful, and sitting in your browser. All you have to do is peel it open and start building.s